## 1. Construct the emission matrix p(w/t)

In [111]:
## Construct bag of word for word and tag seperately
word_count = {}
tag_count = {}
with open ("C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_3_Sequence_Tagging/hw3-data/train.txt") as train_file:
    for line in train_file:
        line_list = line.split()
        for i in range(len(line_list)):
            temp = line_list[i].split('/')
            if temp[0] not in word_count:
                word_count[temp[0]]=1
            if temp[0] in word_count:
                word_count[temp[0]]+=1
            if temp[1] not in tag_count:
                tag_count[temp[1]]=1
            if temp[1] in tag_count:
                tag_count[temp[1]]+=1

#word_count = {}
#{'spoke': 6,
# 'finished': 53,
# 'sucker': 5,
# ...
# ...

#tag_count = {}
# {'A': 3031, 'D': 65382, 'E': 8625, 'F': 78056, 'N': 1333610, 'R': 101735}

In [112]:
## Construct the tree structure to store the c(t,w). 
# Use the dictionary to store the counts
# For the dictionary word_tag_pair_counts, the keys are the tags in the training set, the value is a dictionary, in which the keys 
# are the words, the the value are the counts for this word-key pair 
tag_word_pair_counts = {}
with open ("C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_3_Sequence_Tagging/hw3-data/train.txt") as train_file:
    for line in train_file:
        line_list = line.split()
        for i in range(len(line_list)):
            temp = line_list[i].split('/')
            if temp[1] not in tag_word_pair_counts:
                tag_word_pair_counts[temp[1]] = {}
                tag_word_pair_counts[temp[1]][temp[0]] = 1
            else:
                if temp[0] not in tag_word_pair_counts[temp[1]]:
                    tag_word_pair_counts[temp[1]][temp[0]] = 1
                else:
                    tag_word_pair_counts[temp[1]][temp[0]] += 1 
#tag_word_pair_counts
#{'A': {',': 389,
#  '--': 17,
#  '.': 40,
#  '?': 7,
#  '[inaudible]': 2,
#  'a': 61,
#  ...
#  ...

In [113]:
## Construct the emission matrix, each entry is p(w/t), I use add one smoothing to deal with unknown word

## I use a dictionary of dictionary to represent this matrix
## The key of the outer dictionary is the tag, and the the key of the sub-dictionary is their corresponding word
emission_matrix = {}
delta = 0.1
V = len(word_count) + 1 ## assum only one kind of unseen words
for tag in tag_count:
    emission_matrix[tag]={}
    sum_tag_allword = 0
    for word in tag_word_pair_counts[tag]:
        sum_tag_allword += tag_word_pair_counts[tag][word]
    sum_tag_allword += V*delta ## Add one smoothing
    for word in tag_word_pair_counts[tag]:
        emission_matrix[tag][word] = (tag_word_pair_counts[tag][word]+delta)/sum_tag_allword
    emission_matrix[tag]['unk'] = delta/sum_tag_allword

# emission_matrix
#{'A': {',': 0.12838283828382838,
#  '--': 0.005610561056105611,
#  '.': 0.013201320132013201,
#  '?': 0.0023102310231023103,
#  '[inaudible]': 0.0006600660066006601,
#  ....
#  ....

In [114]:
## Obtain p(you/t)
print ("p(you/A): {}".format(emission_matrix['A']['you']))
print ("p(you/D): {}".format(emission_matrix['D']['you']))
print ("p(you/E): {}".format(emission_matrix['E']['you']))
print ("p(you/F): {}".format(emission_matrix['F']['you']))
print ("p(you/N): {}".format(emission_matrix['N']['you']))
print ("p(you/R): {}".format(emission_matrix['R']['you']))

p(you/A): 0.005543499704083646
p(you/D): 0.2132616434292495
p(you/E): 0.0007596361249179405
p(you/F): 1.3733862711314207e-05
p(you/N): 0.012937615299839477
p(you/R): 0.012692126082892467


## 2. Construct transtion matrix p(t'/t) 

In [115]:
## Considering the start and stop tag, there are 8 different tags ['A', 'D', 'E', 'F', 'N', 'R', <s>, </s>]
## t =  ['A', 'D', 'E', 'F', 'N', 'R', <s>]
## t' = ['A', 'D', 'E', 'F', 'N', 'R', </s>]

## Construct a matrix to store the counts of c(t',t) 
## I use a dictionary of dictionary to represent this matrix
## The key of the outer dictionary is t, and the the key of the sub-dictionary is t'

tag_tag_count = {}
with open ("C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_3_Sequence_Tagging/hw3-data/train.txt") as train_file:
    for line in train_file:
        line_list = line.split()
        temp_tag_list = []
        temp_tag_list.append("<s>")
        for i in range(len(line_list)):
            temp = line_list[i].split('/')
            temp_tag_list.append(temp[1])
        temp_tag_list.append("</s>")
        for i in range(len(temp_tag_list)-1): ## i go through all the t in one sentence
            if temp_tag_list[i] not in tag_tag_count:
                tag_tag_count[temp_tag_list[i]]={}
                tag_tag_count[temp_tag_list[i]][temp_tag_list[i+1]]=1
            else:
                if temp_tag_list[i+1] not in tag_tag_count[temp_tag_list[i]]:
                    tag_tag_count[temp_tag_list[i]][temp_tag_list[i+1]]=1
                else:
                    tag_tag_count[temp_tag_list[i]][temp_tag_list[i+1]]+=1
                    


In [116]:
tag_tag_count

{'<s>': {'A': 28, 'D': 11005, 'E': 1978, 'F': 12612, 'N': 153233, 'R': 11318},
 'A': {'</s>': 46, 'A': 2564, 'D': 19, 'E': 2, 'F': 46, 'N': 305, 'R': 48},
 'D': {'</s>': 4751,
  'A': 38,
  'D': 38134,
  'E': 114,
  'F': 736,
  'N': 18923,
  'R': 2685},
 'E': {'</s>': 186, 'A': 1, 'D': 142, 'E': 5354, 'F': 64, 'N': 2378, 'R': 499},
 'F': {'</s>': 6014,
  'A': 89,
  'D': 1999,
  'E': 83,
  'F': 39789,
  'N': 27764,
  'R': 2317},
 'N': {'</s>': 177367,
  'A': 174,
  'D': 11910,
  'E': 653,
  'F': 20663,
  'N': 1101003,
  'R': 21839},
 'R': {'</s>': 1810,
  'A': 136,
  'D': 2172,
  'E': 440,
  'F': 4145,
  'N': 30003,
  'R': 63028}}

In [117]:
## Construct transition matrix
## Construct the transition matrix to store the p(t'/t). The data structure is the same as that store c(t',t)
transition_matrix = {}
for tag in tag_tag_count:
    transition_matrix[tag] = {}
    sum_temp = 0
    for taggg in tag_tag_count[tag]:
        sum_temp += tag_tag_count[tag][taggg]
    for taggg in tag_tag_count[tag]:
        transition_matrix[tag][taggg] = tag_tag_count[tag][taggg]/sum_temp
    

In [118]:
transition_matrix

{'<s>': {'A': 0.0001472335860843228,
  'D': 0.05786805767349901,
  'E': 0.010401001188385373,
  'F': 0.0663182138462671,
  'N': 0.8057515748735369,
  'R': 0.05951391883222733},
 'A': {'</s>': 0.015181518151815182,
  'A': 0.8462046204620463,
  'D': 0.006270627062706271,
  'E': 0.0006600660066006601,
  'F': 0.015181518151815182,
  'N': 0.10066006600660066,
  'R': 0.015841584158415842},
 'D': {'</s>': 0.07266637096404154,
  'A': 0.0005812086080053838,
  'D': 0.5832581330967712,
  'E': 0.0017436258240161515,
  'F': 0.011257093039262171,
  'N': 0.2894265918233126,
  'R': 0.041066976644590934},
 'E': {'</s>': 0.021567717996289426,
  'A': 0.00011595547309833024,
  'D': 0.016465677179962893,
  'E': 0.6208256029684601,
  'F': 0.0074211502782931356,
  'N': 0.2757421150278293,
  'R': 0.05786178107606679},
 'F': {'</s>': 0.077048235218756,
  'A': 0.001140221638588175,
  'D': 0.025610146691435527,
  'E': 0.0010633527640766126,
  'F': 0.5097559413234258,
  'N': 0.3556979053231696,
  'R': 0.029684197

## 3. Decoding on the test file -- Viterbi Algorithm

In [119]:
import numpy as np

In [120]:
## Viterbi Algorithm Implementation
## Dynamic Programming

tag_list = ['<s>', 'A', 'D', 'E', 'F', 'N', 'R', '</s>']
count_correct_prediction = 0
count_totalword = 0
with open ("C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_3_Sequence_Tagging/hw3-data/test.txt") as test_file:
    for line in test_file:
        word_eachline = []
        tag_eachline = []
        tag_eachline.append("<s>")
        line_list = line.split()
        for item in line_list:
            temp = item.split('/')
            word_eachline.append(temp[0])
            tag_eachline.append(temp[1])
        tag_eachline.append('</s>')
        ## Use Viterbi Algorithm
        ## Viterbi matrix is len(tag_list)*len(word_eachline)
        viterbi = np.zeros((len(tag_list),len(word_eachline))) ## intialize the viterbi matrx all ones 
        backpointer = np.zeros((len(tag_list),len(word_eachline)),dtype=int)
        for i in range(1, len(tag_list)-1):
            ## Get the entries for the first column in Viterbi matrix
            if tag_list[i] in transition_matrix["<s>"] and word_eachline[0] in emission_matrix[tag_list[i]]:
                viterbi[i,0] = transition_matrix["<s>"][tag_list[i]]*emission_matrix[tag_list[i]][word_eachline[0]]
            
            if tag_list[i] not in transition_matrix["<s>"] and word_eachline[0] in emission_matrix[tag_list[i]]:
                viterbi[i,0] = 0
            
            if tag_list[i] in transition_matrix["<s>"] and word_eachline[0] not in emission_matrix[tag_list[i]]:
                viterbi[i,0] = transition_matrix["<s>"][tag_list[i]]*emission_matrix[tag_list[i]]["unk"]
                
            if tag_list[i] not in transition_matrix["<s>"] and word_eachline[0] not in emission_matrix[tag_list[i]]:
                viterbi[i,0] = 0
                
            backpointer[i,0] = 0 ## 0 means the previous tag is '<s>'
            
        ## Calculate other entries
        for i in range(1,len(word_eachline)):
            for j in range(1, len(tag_list)-1):
                temp1 = []
                temp2 = []
                for k in range(1, len(tag_list)-1):
                    if tag_list[j] in transition_matrix[tag_list[k]] and word_eachline[i] in emission_matrix[tag_list[j]]:
                        temp1.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]]*emission_matrix[tag_list[j]][word_eachline[i]])
                        temp2.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]])
                    if tag_list[j] not in transition_matrix[tag_list[k]] and word_eachline[i] in emission_matrix[tag_list[j]]:
                        temp1.append(0)
                        temp2.append(0)
                    if tag_list[j] in transition_matrix[tag_list[k]] and word_eachline[i] not in emission_matrix[tag_list[j]]:
                        temp1.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]]*emission_matrix[tag_list[j]]['unk'])
                        temp2.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]])
                    if tag_list[j] not in transition_matrix[tag_list[k]] and word_eachline[i] not in emission_matrix[tag_list[j]]:
                        temp1.append(0)
                        temp2.append(0)
                viterbi[j,i] = max(temp1)
                backpointer[j,i]= temp2.index(max(temp2))+1  ## here if there are more than one largest values in temp2, the index function only get the smallest index
                
        ## Temination step
        temp = []
        for i in range(1, len(tag_list)-1):
            if "</s>" in transition_matrix[tag_list[i]]:
                temp.append(viterbi[i,-1]*transition_matrix[tag_list[i]]['</s>'])
            else:
                temp.append(0)
        viterbi[tag_list.index("</s>"),-1] = max(temp)
        backpointer[tag_list.index("</s>"),-1] = temp.index(max(temp))+1
        
        ## Backtrack to get the tag sequence
        predict_eachline = ["</s>"]
        predict_eachline.append(tag_list[backpointer[-1,-1]])
        prev = backpointer[-1,-1]

        for i in range(len(word_eachline)-1, 0, -1):
            predict_eachline.append(tag_list[backpointer[prev,i]])
            prev = backpointer[prev,i]
        predict_eachline.append(tag_list[backpointer[prev,0]])
        predict_eachline = predict_eachline[::-1]
        
        ## Count the correct prediction
        for i in range(1,len(predict_eachline)-1):
            count_totalword += 1
            if predict_eachline[i] == tag_eachline[i]:
                count_correct_prediction += 1    

In [121]:
print ("The prediction accuracy by bigram hidden Markov model is : {}.".format(count_correct_prediction/count_totalword))

The prediction accuracy by bigram hidden Markov model is : 0.8898264889011539.


## 4. Prediction on the second line of text file

In [122]:
tag_list = ['<s>', 'A', 'D', 'E', 'F', 'N', 'R', '</s>']
with open ("C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_3_Sequence_Tagging/hw3-data/test.txt") as test_file:
    line = next(test_file)
    line = next(test_file)
    word_eachline = []
    tag_eachline = []
    tag_eachline.append("<s>")
    line_list = line.split()
    for item in line_list:
        temp = item.split('/')
        word_eachline.append(temp[0])
        tag_eachline.append(temp[1])
    tag_eachline.append('</s>')
        ## Use Viterbi Algorithm
        ## Viterbi matrix is len(tag_list)*len(word_eachline)
    viterbi = np.zeros((len(tag_list),len(word_eachline))) ## intialize the viterbi matrx all ones 
    backpointer = np.zeros((len(tag_list),len(word_eachline)),dtype=int)
    for i in range(1, len(tag_list)-1):
            ## Get the entries for the first column in Viterbi matrix
        if tag_list[i] in transition_matrix["<s>"] and word_eachline[0] in emission_matrix[tag_list[i]]:
            viterbi[i,0] = transition_matrix["<s>"][tag_list[i]]*emission_matrix[tag_list[i]][word_eachline[0]]
            
        if tag_list[i] not in transition_matrix["<s>"] and word_eachline[0] in emission_matrix[tag_list[i]]:
            viterbi[i,0] = 0
            
        if tag_list[i] in transition_matrix["<s>"] and word_eachline[0] not in emission_matrix[tag_list[i]]:
            viterbi[i,0] = transition_matrix["<s>"][tag_list[i]]*emission_matrix[tag_list[i]]["unk"]
                
        if tag_list[i] not in transition_matrix["<s>"] and word_eachline[0] not in emission_matrix[tag_list[i]]:
            viterbi[i,0] = 0
                
        backpointer[i,0] = 0 ## 0 means the previous tag is '<s>'
            
        ## Calculate other entries
    for i in range(1,len(word_eachline)):
        for j in range(1, len(tag_list)-1):
            temp1 = []
            temp2 = []
            for k in range(1, len(tag_list)-1):
                if tag_list[j] in transition_matrix[tag_list[k]] and word_eachline[i] in emission_matrix[tag_list[j]]:
                    temp1.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]]*emission_matrix[tag_list[j]][word_eachline[i]])
                    temp2.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]])
                if tag_list[j] not in transition_matrix[tag_list[k]] and word_eachline[i] in emission_matrix[tag_list[j]]:
                    temp1.append(0)
                    temp2.append(0)
                if tag_list[j] in transition_matrix[tag_list[k]] and word_eachline[i] not in emission_matrix[tag_list[j]]:
                    temp1.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]]*emission_matrix[tag_list[j]]['unk'])
                    temp2.append(viterbi[k,i-1]*transition_matrix[tag_list[k]][tag_list[j]])
                if tag_list[j] not in transition_matrix[tag_list[k]] and word_eachline[i] not in emission_matrix[tag_list[j]]:
                    temp1.append(0)
                    temp2.append(0)
            viterbi[j,i] = max(temp1)
            backpointer[j,i]= temp2.index(max(temp2))+1  ## here if there are more than one largest values in temp2, the index function only get the smallest index
                
        ## Temination step
    temp = []
    for i in range(1, len(tag_list)-1):
        if "</s>" in transition_matrix[tag_list[i]]:
            temp.append(viterbi[i,-1]*transition_matrix[tag_list[i]]['</s>'])
        else:
            temp.append(0)
    viterbi[tag_list.index("</s>"),-1] = max(temp)
    backpointer[tag_list.index("</s>"),-1] = temp.index(max(temp))+1

In [123]:
print (viterbi)

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  2.90458840e-09   4.83450480e-08   6.07309950e-11   1.88948169e-11
    8.92130265e-16   7.94645322e-15   1.33573023e-16   9.83355264e-19
    6.39691771e-19   4.86577352e-20   1.14531277e-22   5.18139255e-25
    9.51463828e-29   1.58834699e-33   2.65154184e-38   1.72231592e-39
    3.16270440e-43   9.91168711e-42]
 [  2.66079767e-06   4.45576069e-06   4.23517840e-07   7.78069742e-08
    6.73117035e-14   7.32391327e-13   1.15417129e-13   1.43383701e-14
    2.63418701e-15   1.16222131e-19   3.44717925e-24   4.30430213e-28
    2.64713916e-29   2.17931156e-33   5.53642103e-38   2.76883333e-37
    2.39534939e-43   9.13518707e-40]
 [  9.75429165e-08   1.20307929e-07   2.69241105e

In [124]:
print(backpointer)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 4 4 2 2 4 4 1 2 2 1 1 1 1 1 1 1 4]
 [0 4 4 2 2 4 4 2 2 2 5 5 5 5 5 5 2 4]
 [0 4 4 2 2 4 4 2 2 2 3 5 6 5 5 5 3 4]
 [0 4 4 2 4 4 4 5 2 4 6 5 5 5 5 5 4 4]
 [0 4 4 2 2 4 4 5 2 2 5 5 5 5 5 5 5 4]
 [0 4 6 2 6 4 6 6 2 6 6 6 6 6 5 5 6 4]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4]]


In [125]:
pred_2ndline = ["</s>"]
pred_2ndline.append(tag_list[backpointer[-1,-1]])
prev = backpointer[-1,-1]

for i in range (len(word_eachline)-1, 0, -1):
    pred_2ndline.append(tag_list[backpointer[prev,i]])
    prev = backpointer[prev,i]
pred_2ndline.append(tag_list[backpointer[prev,0]])
pred_2ndline = pred_2ndline[::-1]

In [126]:
print("The prediction on the second line of test file is: {}.".format(pred_2ndline))

The prediction on the second line of test file is: ['<s>', 'F', 'F', 'D', 'F', 'F', 'F', 'D', 'D', 'D', 'N', 'N', 'N', 'N', 'N', 'N', 'F', 'F', 'F', '</s>'].


In [127]:
print("The log probability of this model output is: {}.".format(np.log(viterbi[-1,-1])))

The log probability of this model output is: -89.09235372457093.


In [128]:
pred = []
pred.append(tag_list[backpointer[-1,-1]])
prev = backpointer[-1,-1]

for i in range (len(word_eachline)-1, 0, -1):
    pred.append(tag_list[backpointer[prev,i]])
    prev = backpointer[prev,i]
pred = pred[::-1]

In [129]:
print(pred)

['F', 'F', 'D', 'F', 'F', 'F', 'D', 'D', 'D', 'N', 'N', 'N', 'N', 'N', 'N', 'F', 'F', 'F']


In [130]:
output_label_N = []
for i in range(1,len(pred_2ndline)-1):
    if pred_2ndline[i] == 'N':
        output_label_N.append(word_eachline[i-1])
print (output_label_N)

['i', 'guess', "it's", 'pretty', 'deep', 'feelings']
